In [1]:
import json
import pandas as pd
import unicodedata

def clean_unicode(text):
    return unicodedata.normalize("NFKC", text)

In [5]:
infer_data = pd.read_json('./data/loco_test_conspemollm.json', lines=True)
instruction_list = infer_data.apply(
    lambda row: pd.Series(
        {
            "instruction": clean_unicode(
                f"Human: \n" + row["instruction"] + "\n\nAssistant:\n"
            )
        }
    ),
    axis=1,
)["instruction"].to_list()
print(instruction_list[0])

Human: 
Task: Determine if the text is a conspiracy theory. Classify it into one of the following two classes: 0. non-conspiracy. 1. conspiracy.
Text: Coronavirus is indiscriminate. We have seen young and old affected, rich and poor. In the UK alone, we have seen the Prime Minister himself succumb to it and even Prince Charles. But this pandemic has repercussions beyond our nation states. In a short time, it has paralysed the ‘developed’ world and threatened its very existence.

The unipolar world as we know it is being questioned like never before. Nations demonised in the West – Russia, China and Cuba – have been the very countries sending aid to the West.

Never has one witnessed a more ironic act than the US accepting aid from Russia, its arch geopolitical enemy and ‘the primary threat to American interests’ – according to a survey published last year by a US think-tank. The Antonov An-124-100 cargo plane delivered vital medical equipment and masks from Moscow to the US on Wednesda

In [14]:
# get first row
if 'output' in infer_data.iloc[0]:
    print('yes')

yes


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoConfig

max_new_tokens = 1024
generation_config = dict(
    temperature=0.9,
    top_k=30,
    top_p=0.6,
    do_sample=True,
    num_beams=1,
    repetition_penalty=1.2,
    max_new_tokens=max_new_tokens,
)

load_type = torch.float16  # Sometimes may need torch.float32
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = AutoTokenizer.from_pretrained('./model/ConspEmoLLM-7b')
tokenizer.pad_token_id = 0
tokenizer.bos_token_id = 1
tokenizer.eos_token_id = 2
tokenizer.padding_side = "left"
model_config = AutoConfig.from_pretrained('./model/Emollama-chat-7b')
model = AutoModelForCausalLM.from_pretrained(
    './model/Emollama-chat-7b',
    torch_dtype=load_type,
    config=model_config,
    device_map="auto",
)

if device.type == "cpu":
    model.float()

model.eval()
print("Load model successfully")
print(model.config.max_position_embeddings)

In [4]:
from huggingface_hub import snapshot_download

model_path = snapshot_download(repo_id="lzw1008/ConspEmoLLM-7b", local_dir="./model/ConspEmoLLM-7b")

print("Model downloaded to:", model_path)

Fetching 12 files: 100%|██████████| 12/12 [02:56<00:00, 14.70s/it]

Model downloaded to: /Users/honhaochen/Documents/ConspEmoLLM/model/ConspEmoLLM-7b


In [6]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('./model/ConspEmoLLM-7b')
tokenizer.pad_token_id = 0
tokenizer.bos_token_id = 1
tokenizer.eos_token_id = 2
tokenizer.padding_side = "left"

input_txt = "testing, hello world"
truncated_text = tokenizer.decode(
    tokenizer(input_txt, truncation=True, max_length=1800, add_special_tokens=False)["input_ids"],
    skip_special_tokens=True,
    spaces_between_special_tokens=False,
)
print("Truncated text:", truncated_text)

Truncated text: testing, hello world


both ConspEmoLLM and Emollama-chat-7b has max_position_embeddings of 2048


In [43]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("./model/ConspEmoLLM-7b")
tokenizer.pad_token_id = 0
tokenizer.bos_token_id = 1
tokenizer.eos_token_id = 2
tokenizer.padding_side = "left"

text = "Task: Determine if the text is a conspiracy theory. Classify it into one of the following two classes: 0. non-conspiracy. 1. conspiracy.\nText: Last week, your CNN producer, Matthew Reynard, notified me that CNN is featuring me in a documentary about \u201cvaccine misinformation\u201d. As usual, Mr. Reynard did not point out a single factual assertion by me that was incorrect (I carefully source all of my statements about vaccines to government databases or peer-reviewed publications). CNN uses the term \u201cvaccine misinformation\u201d as a euphemism for any statement that departs from the Government / Pharma orthodoxy that all vaccines are safe, necessary, and effective for all people.\r\n\r\nI have always admired you, Sanjay. Your obvious talents aside, you seem to be genuinely compassionate and to value integrity. Earlier in your career, you showed a courageous willingness to challenge Big Pharma\u2019s vaccine orthodoxies. However, I respectfully point out that CNN and particularly you, Sanjay, are today among the most prolific broadcasters of \u201cvaccine misinformation\u201d. Over the last several years, I cannot recall seeing a single substantial CNN segment on vaccines that did not include easily verified factual misstatements. CNN\u2019s recent special, \u201cPandemic\u201d, was a showcase of erroneous assertions about the flu vaccine. Since I don\u2019t like to think that you deliberately mislead the public\u2014particularly about critical public health choices\u2014I have taken the time to point out some of your most frequent errors.\r\n\r\nI hope you will take time to read this. This critique has special relevance during the current coronavirus crisis, not to mention its important implications for the roles of government and press in a democracy. CNN and other media outlets treat CDC, NIH, and WHO pronouncements as infallible truths. In fact, regulatory capture has made these agencies subsidiaries of Big Pharma, and the lies that CDC has been telling us about flu are now muddying the debate over coronavirus.\r\n\r\n1. CNN assertion: In your annual flu shot promotions, you routinely parrot CDC\u2019s estimates of overall flu deaths which have ranged in recent years from 36,000 for the 1990-1991 flu season to 80,000 for the 2017-2018 flu season.\r\n\r\nFact: The HHS\u2019s mortality and morbidity data\u2014available on the National Center for Health Statistics (NCHS) website\u2014show that CDC\u2019s (and CNN\u2019s) annual estimates are off by orders of magnitude.\r\n\r\nNCHS data report the average number of mortalities attributable to influenza on death certificates is little more than 1,000. CDC devises its inflated estimate by deliberately conflating flu deaths with pneumonia deaths. This device is deceitful since most of these fatalities are unrelated to the flu (and therefore, impervious to flu vaccines). In 2005, the British Medical Journal (BMJ) Editor, Dr. Peter Doshi, published a comprehensive rebuke of CDC\u2019s annual ritual of exaggerating flu mortalities entitled \u201cDissecting CDC\u2019s Deception: Are US Flu Death Figures More PR Than Science?\u201d Doshi accuses the CDC of purposefully inflating flu deaths to frighten the public into purchasing vaccines. To illustrate CDC\u2019s chicanery, Doshi observed that CDC\u2019s announced number of reported pneumonia and influenza deaths in 2001 at 62,034. Yet less than half of one percent of those were actually attributed to influenza. Furthermore, of the mere 257 cases that could reasonably be blamed on the flu in CDC\u2019s mortality data, only 7 percent were laboratory confirmed cases of influenza. That\u2019s 18 lab confirmed influenza cases out of 62,034 \u201cpneumonia and influenza\u201d deaths\u2014or just 0.03 percent, according to HHS\u2019s own National Center for Health Statistics (NCHS).\r\n\r\nSubtracting pneumonia, the true number of influenza-associated deaths from 1979 to 2002 averaged 1,348, according to the NCHS data. CNN routinely reports figures forty times this number.\r\n\r\nDr. Doshi charges the CDC with deliberately lying about annual flu deaths to \u201c[work] in manufacturers\u2019 interest by conducting campaigns to increase flu vaccination\u201d. He warns that \u201cby arbitrarily linking flu with pneumonia, current data are statistically biased.\u201d\r\n\r\nBy faithfully parroting CDC inflated numbers\u2014with no due diligence\u2014CNN has made itself complicit in this annual charade, making it difficult now to accurately assess the relative risk of COVID-19 as compared to flu and, therefore, rationally measure an appropriate response.\r\n\r\n2. CNN assertion: CNN routinely promotes the flu shot for everyone older than 6 months, proclaiming that the best way to protect against serious cases of the ailment \u201cis to get a flu shot\u201d.\r\n\r\nIn reality, there is absolutely no scientific basis for the CDC\u2019s assertion that the influenza vaccine is the most effective way to prevent the flu.\r\n\r\nThe Cochrane Collaboration\u2019s comprehensive 2010 meta-analysis of published influenza vaccine studies found that the influenza vaccination has \u201cno effect\u201d on hospitalization, and that there is \u201cno evidence that vaccines prevent viral transmission or complications.\u201d\r\n\r\nThe Cochrane Researchers concluded in 2010 that the scientific evidence \u201cseem[s] to discourage the utilization of vaccination against influenza in healthy adults as a routine public health measure.\u201d\r\n\r\nFour years later, Cochrane published a follow-up meta-review including dozens of more recent scientific studies and again concluded bluntly that the body of scientific data provides \u201cno evidence for the utilization of vaccination against influenza in healthy adults as a routine public health measure.\u201d\r\n\r\nIn other words, despite CNN\u2019s relentless hectoring, there is no scientific evidence that all the billions of dollars America spends on influenza vaccination each year actually provides any health benefit, much less a net economic benefit\u2014apart from the financial windfall to the four pharmaceutical companies that manufacture these vaccines\u2014and who happen to be among CNN\u2019s top advertisers.\r\n\r\nYou and CNN frequently parrot CDC\u2019s claim that a flu shot reduces the chances that an individual will transmit the flu to others. Pandemic repeated this assertion. CNN offers this supposed benefit as the justification for school vaccine mandates.\r\n\r\nFact: However, in their 2010 systematic meta review of the literature, the Cochrane researchers found \u201cno evidence that vaccines prevent viral transmission or complications\u201d.\r\n\r\nEven more worrisome, a study from January 18, 2018, in the Journal of the Proceedings of the National Academy of Sciences of the United States of America, PNAS, found that influenza vaccination actually increased transmission of the virus, with vaccinated individuals shedding more than six times as much aerosolized virus in their breath than unvaccinated individuals.\r\n\r\nThose scientists were not altogether surprised by this finding explaining that \u201ccertain types of prior immunity\u201d\u2014in this case, the kind of immunity conferred by the vaccine as opposed to naturally acquired immunity\u2014 \u201cpromote lung inflammation, airway closure, and aerosol generation.\u201d They conclude that, \u201cIf confirmed, this observation, together with recent literature suggesting reduced protection with annual vaccination, would have implications for influenza vaccination recommendations and policies.\u201d\r\n\r\nCNN frequently repeats CDC\u2019s advice that children should get the flu shot, which you assure CNN\u2019s audience has been proven safe.\r\n\r\n: A 2012 Cochrane review looking at studies of influenza vaccination in healthy children found no safety studies in children under age two, and declared that safety studies were \u201curgently required\u201d.\r\n\r\n: CNN also promotes CDC\u2019s recommendation that all pregnant women get a flu shot.\r\n\r\n: CDC recommends the flu shot for pregnant women despite the fact that FDA\u2014the agency charged with assessing vaccine safety\u2014has refused to license the flu shot during pregnancy due to grave safety concerns. (Sanjay; I encourage you to confront FDA and make inquiries about this inter-agency conflict.) Every influenza vaccine package insert contains warnings about the lack of safety studies in pregnant women and nursing mothers.\r\n\r\nA 2014 Cochrane review found that the number of randomized, placebo-controlled trials examining the safety and effectiveness of vaccinating pregnant women was zero.\r\n\r\nA 2019 article by Alberto Donzelli in , asks the question, \u201cInfluenza vaccination for all pregnant women?\u201d and argues, \u201cSo far the less biased evidence does not favour it\u201d. Donelli found that public health recommendations on flu shots during pregnancy had systematically overestimated \u201cthe vaccine effectiveness and safety\u201d\u2014and that the published science showed \u201can excess of local adverse effects and a tendency for serious adverse events with uncertain or very limited protection against influenza\u201d. Donzelli observes that flu vaccine trials in Africa and Asia have shown excessive infection and deaths in infants associated with flu shots during pregnancy.\r\n\r\nCNN urges seniors to get their flu shots. CDC credits the vaccine with a dramatic reduction in influenza-related deaths among the elderly.\r\n\r\nFact: The scientific community has thoroughly debunked CDC\u2019s claims that the flu shot reduces death among seniors.\r\n\r\nResearchers from the National Institutes of Health (NIH) ridicule CDC\u2019s mortality claims in a study published in April 2005 in Archives of Internal Medicine (now JAMA Internal Medicine). Those NIH researchers pointed out that, despite a dramatic increase in vaccination coverage among people aged 65 or older\u2014from at most 20 percent before 1980 to 65 percent in 2001\u2014pneumonia and influenza mortality rates \u201crose substantially during this period\u201d.\r\n\r\nThe lead author of the 2005 NIH study, Lone Simonsen, was also coauthor with W. Paul Glezen of a 2006 commentary in the International Journal of Epidemiology that reiterated the problems with the CDC\u2019s claims. \u201cAlthough the vaccination rate for elderly people had increased by as much as 67 percent from 1989 to 1997, there was no evidence that vaccination reduced hospitalizations or deaths. On the contrary, \u201cmortality and hospitalization rates continued to increase rather than decline\u201d.\r\n\r\nThe 2005 NIH study authors commented that this result was \u201csurprising\u201d since vaccination was supposed to be \u201chighly effective at reducing influenza-related mortality\u201d\u2014an assumption underlying CDC policy that \u201chas never been studied in clinical trials\u201d.\r\n\r\nSimilarly, a 2008 review in Virology Journal, observes that contrary to the CDC\u2019s claims of a great beneficial effect on mortality, \u201cinfluenza mortality and hospitalization rates for older Americans significantly increased in the 80s and 90s, during the same time that influenza vaccination rates for elderly Americans dramatically increased.\u201d\r\n\r\nIn a 2013 BMJ commentary, Dr. Doshi asked, \u201cwhat evidence is there that influenza vaccines reduce deaths of older people\u2014the reason the policy was originally created? Virtually none\u2026\u201d This means that influenza vaccines are approved for use in older people despite any clinical trials demonstrating a reduction in serious outcomes.\u201d\r\n\r\n\u201cPerhaps most perplexing,\u201d Doshi added, \u201cis officials\u2019 lack of interest in the absence of good quality evidence.\u201d\r\n\r\n7. CNN assertion: You frequently inform the CNN audience that \u201cthe flu vaccine is safe, and significant side effects are very rare.\u201d\r\n\r\nFact: Actual injury rates are impossible to determine since flu shots are exempt from pre-and-post-marketing placebo studies required of other medicines, and because HHS\u2019s post-marketing surveillance system, the Vaccine Adverse Events Reporting System [VAERS], captures \u201cfewer than 1% of vaccine injuries\u201d according to a 2010 HHS-funded study. Nevertheless, some alarming metrics ought to give you pause when you offer these assurances to millions of viewers; Flu vaccines account for nearly \u00bc of payouts for injuries by the Vaccine Injury Compensation Fund (VICA). The Vaccine Court has paid out nearly $1 billion for injuries and deaths caused by flu shots.\r\n\r\nGSK\u2019s vaccine, Flulaval lists, on its manufacturing inserts, over 45 chronic diseases and adverse reactions that FDA believes may be linked to the vaccine. These include a long menu of immune system, allergic, musculoskeletal, psychiatric, respiratory, skin, vascular, and neurological disease including seizure, paralysis, and syncope.\r\n\r\nAustralian data link the influenza vaccine during the 2009 \u2013 2010 flu season to a 1-in-110 risk in children of having febrile convulsions. The pandemic H1N1 influenza vaccine in Europe was associated with a 1-in-55,000 risk of developing narcolepsy. CDC acknowledges that the Pandemrix flu vaccine is associated with an \u201cincreased risk of narcolepsy\u201d.\r\n\r\nA 2015 meta-analysis published in the journal Vaccine has acknowledged \u201ca small but statistically significant association between influenza vaccines, particularly the pandemic ones, and Guillen-Barre Syndrome (GBS)\u201d.\r\n\r\nA 2004 study in the Journal of the American Medical Association, JAMA, noted that GBS was \u201cthe most frequent neurological condition reported after influenza vaccination to the Vaccine Adverse Events Reporting System (VAERS)\u201d.\r\n\r\nThe 2010 Cochrane meta-analysis chided that the post-mortality studies found that a statistically significant association between the influenza vaccine and GBS \u201cdemonstrate the danger of commencing a large vaccination campaign without adequate harms assessment.\u201d\r\n\r\n8. CNN assertion: On March 5, 2020, you and Anderson Cooper did a \u201cTown Hall\u201d segment, \u201cCorona Facts and Fears\u201d, in which you fervently urged listeners to get the flu shot as the best way to keep healthy during the coronavirus pandemic. According to Anderson, \u201cIf you are concerned about coronavirus, you should get a flu shot\u201d.\r\n\r\nHowever, the only study we have been able to find assessing flu shots and coronavirus is a January 2020 US Pentagon study that found that the flu shot INCREASES the risks from coronavirus by 36%. \u201cReceiving influenza vaccination may increase the risk of other respiratory viruses, a phenomenon known as \u201cvirus interference\u2026\u2019vaccine derived\u2019 virus interference was significantly associated with coronavirus\u2026\u201d\r\n\r\nSanjay, I\u2019ve watched your video assuring the public that getting the flu shot cannot increase one\u2019s chances of getting the flu.\r\n\r\nFact: While that assertion has some meager support from a very small number of studies, the overwhelming weight of published science suggests that getting an annual flu shot can actually increase your risk of both flu and flu-like illnesses.\r\n\r\nOnly about 7 percent to 15 percent of what are called \u201cinfluenza-like illnesses\u201d are actually caused by influenza viruses. Many studies suggest the flu vaccine increases vulnerability to both flu infections and the remaining 85% -93% of non-flu respiratory infections.\r\n\r\nA 2011 study of healthy Australian children published in the Pediatric Infectious Disease Journal found that seasonal flu shots increase the risk of flu by 73% and doubled the risk of non-flu respiratory infections.\r\n\r\nSimilarly, another 2012 randomized controlled trial published in Clinical Infectious Diseases found that influenza-vaccinated children had no significantly lessened risk from influenza and also a higher risk of infection from non-influenza viruses.\r\n\r\nFurthermore, the flu vaccine depletes capacity to fight off future flu infections. In April 2010, a study (by Skowronek, et al) published in the journal PLoS Medicine reported the \u201cunexpected\u201d finding from four epidemiologic studies in Canada that receipt of the influenza vaccine for the 2008 \u2013 2009 season, while apparently effective in reducing the risk of illness due to the seasonal flu, was associated with an increased risk of illness due to the pandemic influenza A (H1N1) \u201cswine flu\u201d virus during the spring and summer of 2009. The scientists suggested that this finding could be due to the difference in the way the vaccine affects the immune system compared with natural infection.\r\n\r\nUnder this hypothesis, repeated vaccination \u201ceffectively blocks the more robust, complex, and cross-protective immunity afforded by prior infection.\u201d\r\n\r\nWhen unvaccinated people are infected with the seasonal influenza virus, they often develop a robust cell-mediated immunity that not only protects against that strain of the virus but is also cross-protective against other strains.\r\n\r\nPeople who\u2019ve annually received the influenza vaccine, on the other hand, \u201cmay have lost multiple opportunities for infection-induced cross-immunity.\u201dThis is because the vaccine is designed to stimulate a strong antibody response, or humoral immunity, but does not confer the same kind of robust cell-mediated immunity as natural infection.\r\n\r\nNIH researchers in their 2005 study also acknowledged the superior effectiveness of naturally acquired immunity at reducing mortality, pointing out that senior citizens who contracted the H3N2 influenza pandemic infection demonstrated a robust immunity in subsequent flu seasons when compared to vaccinated individuals. The sharp decline in influenza-related deaths among people aged 65 to 74 years in the years immediately after the 1968 flu pandemic was most likely due to the acquisition of natural immunity to these viruses (from natural infections).\r\n\r\nAnother study published in 2011 in the Journal of Virology confirmed that annual influenza vaccination indeed hampers the development of a robust cell-mediated immunity. Annual vaccination for influenza, the authors concluded, \u201cmay render young children who have not previously been infected with an influenza virus more susceptible to infection with a pandemic influenza virus of a novel subtype.\u201d\r\n\r\nA 2018 CDC study found there was an increase of acute respiratory infections caused by non-influenza respiratory pathogens following influenza vaccination compared to unvaccinated children during the same period. The authors recommended that potential mechanisms for this association warrant further investigation.\r\n\r\nWhile most studies have looked at only one or two flu seasons, a CDC-funded study published in September 2014 in Clinical Infectious Diseases considered the long-term effects of repeated annual vaccination by looking at five years of vaccination data.\r\n\r\nThe CDC researchers found that the more that people had been vaccinated in prior years, the less effective the vaccine are at preventing the most recent season\u2019s dominant H3N2 virus.\r\n\r\nAs they put it, \u201cvaccine-induced protection was greatest for individuals not vaccinated during the prior 5 years.\u201d\r\n\r\nEssentially, the immune system remembers the original infection and puts out a rapid defense against it, at the expense of developing a new but more appropriate response specifically to the currently infecting strain.\r\n\r\nThe CDC scientists warned that their data \u201craises relevant questions about the potential interference of repeated annual influenza vaccination and possible residual protection from previous season vaccination\u201d; the authors called for further studies.\r\n\r\n10. CNN assertion: One final observation about a different vaccine; In CNN\u2019s regular promotion of measles vaccines, CNN and Sanjay frequently claim that natural measles mortalities are 1-2 in 1000. Those estimates seem calculated to frighten people into taking a measles shot and to drive MMR mandates.\r\n\r\nFact: CDC\u2019s 1963 mortality and morbidity data show that prior to the introduction of the measles vaccine, improvements in nutrition and hygeine had already driven US measles mortality in U.S. to 400 per year, a population ratio of 1/500,000 and a death-case ratio of 1 in 10,000\u2014about the same risk of dying from a lightning strike. Most of those mortalities were among malnourished children, many of whom suffered from intellectual disabilities. The best evidence suggests that measles mortalities would have continued to drop with the introduction of food stamps, W.I.C, and other childhood nutritional programs passed during the War on Poverty after 1964 to relieve hunger in impoverished communities.\r\n\r\nIn their 2010 meta-analysis, the Cochrane researchers accused the CDC of deliberately misrepresenting the science in order to support their universal influenza vaccination recommendation. Nevertheless, CNN continually broadcasts CDC pronouncements as gospel and, ironically, ridicules those of us who actually read the science as \u201cpurveyors of \u2018vaccine misinformation\u2019\u201d.\r\n\r\nMultiple comprehensive federal investigations and whistleblower declarations have documented the corrupt relationship between the CDC\u2019s Vaccine Branch and the four vaccine makers: Merck, Pfizer, Sanofi, and GSK. These include a 2000 report by the US Congress Government Oversight Committee , a 2009 report by the Federal HHS Inspector General, a 2014 letter by David Wright, Director of HHS Office of Research Integrity, and a 2011 letter to Carmen S. Villar, chief of staff for Tom Frieden, from an organization of CDC scientists calling itself \u201cSPIDER\u201d.\r\n\r\nIn 2014, CDC\u2019s senior vaccine safety scientist, Dr. William Thompson, a 17-year CDC veteran, who continues to work for CDC, confessed in a series of depositions, and public and private statements, that his CDC bosses in CDC\u2019s Immunization Branch had systematically ordered him and other researchers to destroy data and falsify study outcomes to hide CDC research linking vaccines to the exploding epidemic of childhood chronic diseases including autism. Doesn\u2019t the abundant evidence of corruption at the Federal health agencies amplify CNN\u2019s obligation to treat government pronouncements with skepticism?\r\n\r\nAt a 2004 workshop for the Institute of Medicine, CDC unveiled a blueprint for the agency\u2019s annual campaigns of fear and deception in a PowerPoint entitled \u201c\u2018Recipe\u2019 for Fostering Public Interest and High Vaccine Demand\u201d. CDC\u2019s in-house P.R. flack Glen Nowak explained that it was necessary to use fear marketing to sell vaccines. CDC\u2019s campaign called for encouraging television medical experts (like Sanjay and Elizabeth Cohen) to \u201cstate concern and alarm\u201d about \u201cand predict dire outcomes\u201d from the flu season. To inspire the necessary terror, the CDC planned to encourage its tame journalists to describe each season as \u201cvery severe\u201d, \u201cmore severe than last or past years\u201d, and \u201cdeadly\u201d. CDC\u2019s press flacks stressed that it was especially important to inspire \u201cconcern, anxiety, and worry\u201d among young, healthy adults who don\u2019t regard the flu with sufficient dread.\r\n\r\nAs the CDC bluntly stated it, \u201cHealth literacy is a growing problem\u201d. In other words, the CDC considers it to be a problem that people are increasingly doing their own research and becoming more adept at educating themselves about health-related issues; Why? Because people who do their own research, read the science, and make informed choices rather than blindly following the CDC\u2019s recommendations are less likely to get the flu shot.\r\n\r\n\u201cDrug companies\u201d, Dr. Doshi observes, \u201chave long known that to sell some products, you would have to first sell people on the disease.\u201d Only, in the case of the influenza vaccine, Doshi adds, \u201cthe salesmen are public health officials\u201d. These public health officials have, in turn, transformed trusted journalists and television doctors into Pharma marketing reps.\r\n\r\nCNN likes to portray CDC\u2019s annual flu shot campaigns as an important public health ritual. The peer-reviewed science exposes CDC\u2019s campaigns as a mercantile propaganda project that is costly and may be injuring public health. CNN can fault CDC officials as the source of its \u201cvaccine misinformation\u201d. But this is a weak gesture. \u201cPeople in power lie\u201d, my father once told me. The function of journalism is to apply scrutiny and skepticism to the pronouncements of government officials and powerful corporations.\r\n\r\nFinally, Sanjay, you and Anderson Cooper often comment with dismay on the monumental tragedy, for our democracy, of having a president who habitually lies. But presidents come and go; the more enduring tragedy, arguably, is that we cannot trust our news media to tell us the truth about vital health issues when advertising dollars are at stake. You scratch your head and wonder how all those Trump supporters don\u2019t share your indignation at President Trump\u2019s mendacity. One answer is that they are disheartened by once-trusted media outlets who have also set the precedent of routinely lying and violating the public trust, wounding in the process our democracy, public faith in critical institutions, and the health of our children.\r\n\r\nP.S. Just as a reminder, here is a 60 Minutes program from over 30 years ago. This is what journalism looked like before Pharma purchased the media.\r\n\r\nCopyright 2020 \u2013 permission to reprint fully granted, WITH links to original story\nClass:\n0"
original_ids = tokenizer(text, add_special_tokens=False)
print("Original IDs:", original_ids['input_ids'])
print(len(original_ids['input_ids']))

text2 = "text " * 7000
test_ids = tokenizer(text2, add_special_tokens=False)
print("Test IDs:", test_ids['input_ids'])

Original IDs: [9330, 29901, 5953, 837, 457, 565, 278, 1426, 338, 263, 21588, 29875, 10068, 6368, 29889, 4134, 1598, 372, 964, 697, 310, 278, 1494, 1023, 4413, 29901, 29871, 29900, 29889, 1661, 29899, 535, 1028, 29875, 10068, 29889, 29871, 29896, 29889, 21588, 29875, 10068, 29889, 13, 1626, 29901, 9208, 4723, 29892, 596, 29696, 14297, 29892, 22292, 28555, 538, 29892, 451, 2164, 592, 393, 29696, 338, 23425, 592, 297, 263, 1842, 653, 1048, 1346, 29894, 5753, 457, 3984, 19678, 8643, 1094, 9670, 29892, 3237, 29889, 28555, 538, 1258, 451, 1298, 714, 263, 2323, 2114, 950, 28306, 491, 592, 393, 471, 10240, 313, 29902, 16112, 2752, 599, 310, 590, 9506, 1048, 325, 5753, 1475, 304, 5874, 21218, 470, 23533, 29899, 27828, 287, 25964, 467, 29696, 3913, 278, 1840, 1346, 29894, 5753, 457, 3984, 19678, 30024, 408, 263, 11878, 561, 331, 1608, 363, 738, 3229, 393, 1401, 5708, 515, 278, 10354, 847, 1963, 22824, 14219, 17324, 29891, 393, 599, 325, 5753, 1475, 526, 9109, 29892, 5181, 29892, 322, 11828, 363,

In [47]:
import torch
import unicodedata

def clean_unicode(text):
    return unicodedata.normalize("NFKC", text)

print(clean_unicode(text))
original_ids = tokenizer(clean_unicode(text), add_special_tokens=False)
print(len(original_ids['input_ids']))

t = torch.tensor(original_ids['input_ids'])
has_nan = torch.isnan(t).any()
print("Contains NaN:", has_nan.item())  # True or False
# print(original_ids)

Task: Determine if the text is a conspiracy theory. Classify it into one of the following two classes: 0. non-conspiracy. 1. conspiracy.
Text: Last week, your CNN producer, Matthew Reynard, notified me that CNN is featuring me in a documentary about “vaccine misinformation”. As usual, Mr. Reynard did not point out a single factual assertion by me that was incorrect (I carefully source all of my statements about vaccines to government databases or peer-reviewed publications). CNN uses the term “vaccine misinformation” as a euphemism for any statement that departs from the Government / Pharma orthodoxy that all vaccines are safe, necessary, and effective for all people.

I have always admired you, Sanjay. Your obvious talents aside, you seem to be genuinely compassionate and to value integrity. Earlier in your career, you showed a courageous willingness to challenge Big Pharma’s vaccine orthodoxies. However, I respectfully point out that CNN and particularly you, Sanjay, are today among 

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import unicodedata
import pandas as pd
def clean_unicode(text):
    return unicodedata.normalize("NFKC", text)

generation_config = dict(
    temperature=0.9,
    top_k=30,
    top_p=0.6,
    do_sample=True,
    num_beams=1,
    repetition_penalty=1.2,
    max_new_tokens=1024,
)

model_name_or_path = "./model/ConspEmoLLM-7b"  # Replace with your actual model path
load_type = torch.float32
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
tokenizer.pad_token_id = 0
tokenizer.bos_token_id = 1
tokenizer.eos_token_id = 2
tokenizer.padding_side = "left"
model = AutoModelForCausalLM.from_pretrained(model_name_or_path, torch_dtype=load_type, device_map='auto')
model.eval()
# Prepare prompt
infer_data = pd.read_json('./data/long_sentence.json', lines=True)
instruction_list = infer_data.apply(
    lambda row: pd.Series(
        {'instruction': clean_unicode(f"Human: \n" + row['instruction'] + "\n\nAssistant:\n")}
    ), axis=1
)['instruction'].to_list()

def main():
    inputs = tokenizer(instruction_list, return_tensors="pt",padding=True)
    input_ids = inputs.input_ids.to(device)
    attention_mask = inputs.attention_mask.to(device)
    print(len(input_ids[0]))
    # Generate response
    with torch.no_grad():
        output = model.generate(
            input_ids = input_ids,
            attention_mask = attention_mask,
            **generation_config
        )
    response = tokenizer.decode(output[0], skip_special_tokens=True, spaces_between_special_tokens=False)

main()

# 5923 (23000), 5947 (23090) | 5949 (23100)
# when on 5947 (23090) words, it becomes unstable. sometimes work, sometimes don't
